In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import lmdb
import json
from datetime import datetime, timezone
import numpy as np
import os
import pandas as pd
import plotly.express as px
from purple_py.log import logger
from purple_py.db import (
    load_events_into_weaviate,
    load_neo4j_data,
    create_weaviate_user_class,
    create_weaviate_event_class,
    add_npub_mean_vec,
    get_summary_text,
)
from purple_py.embed import get_mean_vectors
from purple_py.query import query_weaviate, search_weaviate, filter_users, get_user_uuid
from purple_py.utils import parse_datetime, parse_datetime_str
from sentence_transformers import SentenceTransformer
import weaviate
from nostr_sdk import Keys, Client, Filter, Options, PublicKey, Timestamp
from dotenv import load_dotenv
load_dotenv()
from purple_py.parse import parse_event_json
from purple_py.utils import get_event_bech, get_npub

client = weaviate.Client(
    url="http://localhost:8080",
)


In [ ]:
response = (
    client.query
    .aggregate("Event")
    .with_group_by_filter(["kind"])
    .with_fields("groupedBy { value }")
    .with_meta_count()
    .do()
)

In [ ]:
response

In [ ]:
response = (
    client.query
    .aggregate("Event")
    .with_group_by_filter(["kind"])
    .with_fields("groupedBy { value }")
    .with_meta_count()
    .do()
)
events = response['data']['Aggregate']['Event']
kind_count_list = [{'kind': event['groupedBy']['value'], 'count': event['meta']['count']} for event in events]

df = pd.DataFrame(kind_count_list)

In [ ]:
output = query_weaviate(client, kind=31923)

In [ ]:
len(output)

In [ ]:
response

In [ ]:
created_at_list = get_summary_text(client)

In [ ]:
datetime.utcfromtimestamp(int(np.percentile(created_at_list, 25))), datetime.utcfromtimestamp(int(np.percentile(created_at_list, 75)))

In [ ]:
np.percentile(dates, 25)

In [ ]:
dates = [datetime.utcfromtimestamp(ts) for ts in created_at_list]
lower_date = np.percentile(dates, 25))
upper_date = np.percentile(dates, 75))

df = pd.DataFrame({'Date': dates})

# Create the box plot
fig = px.box(df, y='Date')

fig.update_yaxes(range=["2022-01-01", "2022-12-31"])

# Customize the layout
fig.update_layout(
    title='Box Plot of Timestamps',
    yaxis_title='Date'
)

# Show the plot
fig.show()

In [ ]:
df.Date.median()

In [ ]:
created_at_list[0]

In [ ]:
parse_datetime_str(round(start_date)), parse_datetime_str(round(end_date))

In [ ]:
len(out)

In [ ]:
parse_datetime_str(1652076009)

In [ ]:
out[:2]

In [ ]:
response = client.get_meta()
print(json.dumps(response, indent=2))

In [ ]:
start_time = datetime.now()
print("Start time:", start_time.strftime("%Y-%m-%d %H:%M:%S"))

add_npub_mean_vec(client)

end_time = datetime.now()
duration = end_time - start_time
print("End time:", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duration:", duration)


In [ ]:
df_mean.index[26]

In [ ]:
pubkey ="ffca54e078fd9884a745d70eb0159c0b8a9e7d383a4906a7484b83ebf6101dd5"
response = (
    client.query.get("User", ["pubkey, name"])
    .with_near_object({"id": get_user_uuid(client, pubkey)})
    .with_limit(5)
    .with_additional(["distance"])
    .do()
)

df_similar = pd.DataFrame(response["data"]["Get"]["User"])
df_similar["distance"] = df_similar["_additional"].apply(
    lambda x: x["distance"] if isinstance(x, dict) else None
)
df_similar["npub"] = df_similar["pubkey"].apply(get_npub)
df_similar = df_similar.sort_values("distance", ascending=True).drop(columns=["_additional"])

In [ ]:
df_similar

In [ ]:
similar_data = [
    {
        "npub": f"[{record}](https://njump.me/{record})",
    }
    for record in df_similar["npub"]
]

In [ ]:
similar_data

In [ ]:
df_similar

In [ ]:
df_similar[['pubkey', 'name']].to_dict('records')

In [ ]:
similar_data = [
    {
        "npub": f"[{record['npub']}](https://njump.me/{record['npub']})",
    }
    for record in df_similar.iterrows()
]

In [ ]:
df = pd.DataFrame(response["data"]["Get"]["User"])
df['distance'] = df['_additional'].apply(lambda x: x['distance'])


In [ ]:
df.pubkey.apply(get_npub)

In [ ]:
df.npub.iloc[1]

In [ ]:
response["data"]["Get"]["User"]

In [ ]:
npub="npub1t3ggcd843pnwcu6p4tcsesd02t5jx2aelpvusypu5hk0925nhauqjjl5g4"

In [ ]:
df_filter = filter_users(client, min_num_events=5)

In [ ]:
len(df_filter.pubkey.unique())

In [ ]:
df_mean = get_mean_vectors(df_filter)

In [ ]:
df_mean

In [ ]:
df_mean.head()

In [ ]:
df_filter = filter_users(client, min_num_events=5)
df_vec = df_filter.groupby("pubkey")["vector"].apply(
    lambda x: np.mean(x.tolist(), axis=0)
)

In [ ]:
npub = df_vec.index[0]
type(df_vec.loc[npub])

In [ ]:
for pubkey in df_vec.index:
    print(pubkey)
    break

In [ ]:
vec = df_filter.groupby('pubkey').get_group('e417ee3d910253993ae0ce6b41d4a24609970f132958d75b2d9b634d60a3cc08')['vector'].tolist()
mean_vec = np.mean(vec, axis=0)


In [ ]:
len(mean_vec)

In [ ]:
where_filter = {
  "valueInt": 5,
  "operator": "GreaterThanEqual",
  "path": ["hasCreated"]
}

response = (
  client.query
  .get("User", ["pubkey", "hasCreated {... on Event { event_id, content _additional { vector } }}"])
  .with_where(where_filter)
  .with_limit(10000)
  .do()
)

data = []

# Iterate over each user in the response
for user in response['data']['Get']['User']:
    pubkey = user['pubkey']
    
    # Check if the user has created any events
    if 'hasCreated' in user:
        # Iterate over each event created by the user
        for event in user['hasCreated']:
            event_id = event.get('event_id', '')
            content = event.get('content', '')
            vector = event.get('_additional', {}).get('vector', [])

            # Append the data to the list
            data.append({'pubkey': pubkey, 'event_id': event_id, 'content': content, 'vector': vector})

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data)


In [ ]:
len(df)

In [ ]:
len(df.pubkey.unique())

In [ ]:
response['data']['Get']['User'][0].keys()

In [ ]:
response = search_weaviate(client, "cycling")

In [ ]:
graphql_query = """
{
    Aggregate {
        User {
            meta {
                count
            }
        }
    }
}
"""
result = client.query.raw(graphql_query)
if "errors" in result:
    print("Error in GraphQL Query:", result["errors"])
user_count = result["data"]["Aggregate"]["User"][0]["meta"]["count"]

In [ ]:
user_count

In [ ]:
all_users = get_all_user_pubkeys_paginated(client)

In [ ]:
len(all_users)

In [ ]:
result

In [ ]:
response = (
    client.query.aggregate("User")
    .with_group_by_filter(["pubkey"])
    .with_meta_count()
    .do()
)

In [ ]:
len(response['data']['Aggregate']['User'])

In [ ]:
# response = (
#     client.query.aggregate("User")
#     .with_group_by_filter(["pubkey"])  # Grouping by user's public key
#     .with_fields("hasCreated { meta { count } }")  # Counting the number of events created
#     .with_meta_count()
#     .do()
# )
# response = (
#     client.query.aggregate("User")
#     .with_fields("count")
#     .with_fields("hasCreated { count }")  # Count the number of events created by each user
#     .with_meta_count()
#     .do()
# )

# Fetch all users
users_response = (
    client.query.get("User", ["pubkey"])
    .with_limit(10000)  # Adjust the limit based on your expected number of users
    .do()
)

if "errors" in users_response:
    print("Error fetching users:", users_response["errors"])
    return None

# Extract user pubkeys
user_pubkeys = [user["pubkey"] for user in users_response["data"]["Get"]["User"]]

# Now fetch count of hasCreated for each user
user_event_counts = []
for pubkey in user_pubkeys:
    events_count_response = (
        client.query.aggregate("Event")
        .with_where({"path": ["createdBy"], "operator": "Equal", "valueString": pubkey})
        .with_fields("count")
        .do()
    )

    if "errors" in events_count_response:
        print(f"Error fetching event count for user {pubkey}:", events_count_response["errors"])
        continue

    event_count = events_count_response["data"]["Aggregate"]["Event"]["count"]
    user_event_counts.append({"pubkey": pubkey, "event_count": event_count})

# Convert to DataFrame or your preferred data structure



In [ ]:
response

In [ ]:
def filter_users_debug(client, num_events=5):
    # Step 1: Aggregate users
    graphql_query = f"""
    {{
        Aggregate {{
            User(
                where: {{
                    operator: GreaterThan,
                    path: ["hasCreated"],
                    valueInt: {num_events}
                }}
            ) {{
                groupBy: ["pubkey"]
            }}
        }}
    }}
    """

    # Execute the query
    result = client.query.raw(graphql_query)

    # Print the raw result for debugging
    print("GraphQL Query Result:", result)

    # Check if the result contains errors
    if "errors" in result:
        print("Error in GraphQL Query:", result["errors"])
        return None

    # Proceed with your processing...
    # ...

    return result  # or some processed form of the result


In [ ]:
df_users = filter_users_debug(client)

In [ ]:
df_users = filter_users_nb(client)

In [ ]:
df_users

In [ ]:
# results = query_weaviate(client, npub="your_npub_value", kind="your_kind_value")
response = query_weaviate(client, npub="npub1t3ggcd843pnwcu6p4tcsesd02t5jx2aelpvusypu5hk0925nhauqjjl5g4")
# 



In [ ]:
# text_kinds = [1, 31922, 31923]
cal = query_weaviate(client, kind="31923")

In [ ]:
cal.iloc[0]['event_id']

In [ ]:
lan = search_weaviate(client, "Yondar Dev Call", limit=20)

In [ ]:
lan[lan['content'].str.contains('yondar', na=False, case=False)]


In [ ]:
data = response.to_dict('records')

In [ ]:
data[0]

In [ ]:
len(response)

In [ ]:
# client.schema.delete_class('User')
# client.schema.delete_class('Event')


In [ ]:
# define Event class before User
create_weaviate_event_class(client)
create_weaviate_user_class(client)

In [ ]:
start_time = datetime.now()
print("Start time:", start_time.strftime("%Y-%m-%d %H:%M:%S"))

load_events_into_weaviate(client)

end_time = datetime.now()
duration = end_time - start_time
print("End time:", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duration:", duration)


In [ ]:
# get event by event id
# get all events for pubkey

event_query = {
  "where": {
    "operator": "Equal",
    "path": ["event_id"],
    "valueString": event_id
  }
}

event_response = (
    client.query
    .get("Event", ["created_at", "pubkey", "kind"])
    .with_where(event_query["where"])
    .do()
)
event = event_response['data']['Get']['Event']

In [ ]:
# get all events for pubkey
pubkey = "17538dc2a62769d09443f18c37cbe358fab5bbf981173542aa7c5ff171ed77c4"  # elsat

pubkey_query = {
  "where": {
    "operator": "Equal",
    "path": ["pubkey"],
    "valueString": pubkey
  }
}

user_response = (
    client.query
    .get("User", ["pubkey"])
    # .get("User", [
    #     "hasCreated { ... on Event { event_id created_at pubkey kind content } }"
    # ])
    # .with_where(pubkey_query["where"])
    .do()
)

event_response = (
    client.query
    .get("Event", ["event_id"])
    # .get("User", [
    #     "hasCreated { ... on Event { event_id created_at pubkey kind content } }"
    # ])
    # .with_where(pubkey_query["where"])
    .do()
)

created_response = (
    client.query
    .get("User", [
        "hasCreated { ... on Event { event_id created_at pubkey kind content } }"
    ])
    .do()
)

# Extracting the events associated with the user
pubkey_events = user_response['data']['Get']['User']
events_output = event_response['data']['Get']['Event']
created_output = created_response['data']['Get']['User']


In [ ]:
len(pubkey_events), len(events_output),  len(created_output)

In [ ]:
created_output

In [ ]:
# 1st pass: get all content > min_length, get embeddings
start_time = datetime.now()
print("Start time:", start_time.strftime("%Y-%m-%d %H:%M:%S"))

env = lmdb.open(path=strfry_path, max_dbs=10)
payload_db = env.open_db(b"rasgueadb_defaultDb__EventPayload")
id_db = env.open_db(b"rasgueadb_defaultDb__Event__id")

# record_count = 0
# pubkey_dict = {}
# event_dict = {}

text_kinds = [1, 31922, 31923]
content_list = []
event_id_list = []
client.batch.configure(batch_size=batch_size)
with client.batch as batch:
    with env.begin(db=id_db) as txn:
        cursor = txn.cursor(db=id_db)
        for key, value in cursor:
            pl = txn.get(value, db=payload_db)
            if pl is None:
                print(key, value)
                continue
            event_hex = key.hex()[:64]  # event_id
            event_json = json.loads(pl[1:].decode("utf-8"))
            if "content" not in event_json or len(event_json['content']) < min_content_length:
                continue
            if event_json["kind"] in text_kinds:
                event_id = event_json["id"]
                content = event_json['content']
                if event_id not in event_id_list:
                    content_list.append(content)
                    event_id_list.append(event_id)
end_time = datetime.now()
print("End time:", end_time.strftime("%Y-%m-%d %H:%M:%S"))

# Calculate and print the duration
duration = end_time - start_time
print("Duration:", duration)


In [ ]:
start_time = datetime.now()
print("Start time:", start_time.strftime("%Y-%m-%d %H:%M:%S"))

end_time = datetime.now()
duration = end_time - start_time
print("End time:", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duration:", duration)


In [ ]:
# 1st pass: get all content > min_length, get embeddings
content_list = []
event_id_list = []
offset = 0
while True:
    events_response = client.query.get(
        "Event",
        ["event_id", "content"]
        # ["event_id", "content", "_additional { id }"]
    ).with_limit(page_limit).with_offset(offset).do()

    events = events_response["data"]["Get"]["Event"]
    if events is None:
        break
    for event in events:
        event['event_id']
        content = event['content']
        if len(content) > min_content_length:
            content_list.append(event['content'])
            event_id_list.append(event['event_id'])
    if len(events) < page_limit:
        break
    offset += page_limit

In [ ]:
users = client.query.get("User", ["name", "pubkey", "hasCreated"]).do()
for user in users["data"]["Get"]["User"]:
    print(f"User: {user['name']} has created events: {user['hasCreated']}")


In [ ]:
event["_additional"]["id"]

In [ ]:
user_response["data"]["Get"]["User"][:5]

In [ ]:
embeddings = embedding_model.encode(content_list)

In [ ]:
embeddings[0]

In [ ]:
len(embeddings)

In [ ]:
len(content_list), len(event_id_list)

In [ ]:
# query pubkeys with > 5 events
offset = 0
event_counts = {}
while True:
    events_response = client.query.get(
        "Event",
        # ["event_id", "content", "pubkey"]
        ["event_id", "content", "pubkey", "_additional { id }"]
    ).with_limit(page_limit).with_offset(offset).do()

    events = events_response["data"]["Get"]["Event"]
    if events is None:
        break
    for event in events:
        if len(event['content']) > min_content_length:
            pubkey = event['pubkey']
            event_counts[pubkey] = event_counts.get(pubkey, 0) + 1
    if len(events) < page_limit:
        break
    offset += page_limit

pubkeys_with_events = [
    pubkey for pubkey, count in event_counts.items() if count >= min_num_events
]

print(pubkeys_with_events)


In [ ]:
response = (
    client.query
    .get("Event", ["event_id", "pubkey", "kind", "content"])
    .with_limit(1)
    # .with_additional("vector")
    .with_additional(["distance", "id"])
    .do()
)
print(json.dumps(response, indent=4))

In [ ]:
response = (
    client.query
    .aggregate("Event")
    .with_meta_count()
    .do()
)

print(json.dumps(response, indent=2))

In [ ]:
near_text_config = {
  "concepts": ["hair typo"],
}

response = (
    client.query
    .get("Event", ["event_id", "content"])
    .with_near_text(near_text_config)
    .with_limit(100)
    .with_additional(["distance", "id"])
    .do()
)

result = response['data']['Get']['Event']
# print(json.dumps(response, indent=4))

In [ ]:
pubkey_value = "eab0e756d32b80bcd464f3d844b8040303075a13eabc3599a762c9ac7ab91f4f"

query = """
{
  Get {
    Event(
      where: {
        path: ["pubkey"]
        operator: Equal
        valueString: "%s"
      }
      limit: 10000
    ) {
      event_id
      created_at
      pubkey
      kind
      content
      _additional {
        vector
      }
    }
  }
}
""" % pubkey_value

result = client.query.raw(query)
events = result['data']['Get']['Event']

In [ ]:
vectors = [event['_additional']['vector'] for event in events]
vectors_array = np.array(vectors)
mean_vector = np.mean(vectors_array, axis=0)

# event_ids = [event['event_id'] for event in events]

# # Create a DataFrame
# df_vec = pd.DataFrame(vectors, index=event_ids, columns=[f'feature_{i}' for i in range(len(vectors[0]))])


In [ ]:
mean_vector[:5]

In [ ]:
response = (
    client.query
    .get("Event", ["question"])
    .do()
)

In [ ]:
def process_month(event_json):
    date = datetime.fromtimestamp(event_json['created_at'])
    month = date.strftime("%Y-%m")
    return month

def process_date(event_json):
    date = (
        datetime.fromtimestamp(event_json['created_at'])
        .strftime("%m-%d-%Y")
    )
    return date

def process_kind(event_json):
    return event_json['kind']

def process_db(process_fn):
    env = lmdb.open(path=os.getenv("STRFRY_DB_FOLDER"), max_dbs=10)
    payload_db = env.open_db(b"rasgueadb_defaultDb__EventPayload")
    id_db = env.open_db(b"rasgueadb_defaultDb__Event__id")
    output_list = []

    with env.begin(db=id_db) as txn:
        cursor = txn.cursor(db=id_db)
        for key, value in cursor:
            pl = txn.get(value, db=payload_db)
            if pl is None:
                print(key, value)
                continue
            # event_hex = key.hex()[:64]  # event_id
            event_json = json.loads(pl[1:].decode("utf-8"))
            output_list.append(process_fn(event_json))
    return output_list

In [ ]:
month_list = process_db(process_month)
event_counts = pd.Series(month_list).value_counts().sort_index()
fig = px.bar(event_counts, x=event_counts.index, y=event_counts.values, labels={'x': 'Month', 'y': 'Number of Events'})
fig.update_layout(title='Number of Events by Month', xaxis_title='Month', yaxis_title='Number of Events')
fig.show()

In [ ]:
date_list = process_db(process_date)
event_counts = pd.Series(date_list).value_counts().sort_index()
fig = px.bar(event_counts, x=event_counts.index, y=event_counts.values, labels={'x': 'Date', 'y': 'Number of Events'})
fig.update_layout(title='Number of Events by Date', xaxis_title='Date', yaxis_title='Number of Events')
fig.show()

In [ ]:
kind_list = process_db(process_kind)
counts = pd.Series(kind_list).value_counts().sort_index()
fig = px.bar(counts, x=list(map(str, counts.index)), y=counts.values, labels={'x': 'kind', 'y': 'Number of Events'})
fig.update_layout(title='Number of Events by kind', xaxis_title='kind', yaxis_title='Number of Events')
fig.show()
# 1, 30023


In [ ]:
start_time = datetime.now()
print("Start time:", start_time.strftime("%Y-%m-%d %H:%M:%S"))

end_time = datetime.now()
print("End time:", end_time.strftime("%Y-%m-%d %H:%M:%S"))

# Calculate and print the duration
duration = end_time - start_time
print("Duration:", duration)

In [ ]:
env = lmdb.open(path=os.getenv("STRFRY_DB_FOLDER"), max_dbs=10)
payload_db = env.open_db(b"rasgueadb_defaultDb__EventPayload")
id_db = env.open_db(b"rasgueadb_defaultDb__Event__id")


pubkey_counts = {}
with env.begin(db=id_db) as txn:
    with env.begin(db=payload_db) as tpl:
        for key, value in txn.cursor():
            pl = tpl.get(value)
            if pl is None:
                raise Exception("db corrupt!?")
            event_hex = key.hex()[:64]  # event_id
            event_json = json.loads(pl[1:].decode("utf-8"))  # event json
            pubkey = event_json['pubkey']
            if pubkey in pubkey_counts:
                pubkey_counts[pubkey] += 1
            else:
                pubkey_counts[pubkey] = 1

In [ ]:
top_pubkeys = sorted(pubkey_counts, key=pubkey_counts.get, reverse=True)[:5]

for pubkey in top_pubkeys:
    print(f"PubKey: {pubkey}, Count: {pubkey_counts[pubkey]}")